In [18]:
import pandas as pd

train = pd.read_csv(r"train_final.csv")

print(f'train set은 {train.shape[1]} 개의 feature를 가진 {train.shape[0]} 개의 데이터 샘플로 이루어져 있습니다.')

train.sample(5)

train set은 24 개의 feature를 가진 2999 개의 데이터 샘플로 이루어져 있습니다.


,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
917,918,Male,Loyal Customer,26,Business travel,Business,2823,1,1,1,...,3,5,3,4,3,5,3,38,23,1
1523,1524,Female,Loyal Customer,46,Business travel,Business,2405,2,2,2,...,3,3,5,5,4,5,4,181,176,1
1393,1394,Female,disloyal Customer,9,Business travel,Eco,2154,1,1,1,...,1,2,1,4,2,4,1,0,0,0
2186,2187,Male,Loyal Customer,36,Business travel,Business,1287,4,4,4,...,5,5,5,5,4,5,5,0,0,1
2834,2835,Male,Loyal Customer,31,Personal Travel,Eco,2073,3,2,3,...,3,4,4,3,1,3,3,0,0,0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   id                                 2999 non-null   int64 
 1   Gender                             2999 non-null   object
 2   Customer Type                      2999 non-null   object
 3   Age                                2999 non-null   int64 
 4   Type of Travel                     2999 non-null   object
 5   Class                              2999 non-null   object
 6   Flight Distance                    2999 non-null   int64 
 7   Seat comfort                       2999 non-null   int64 
 8   Departure/Arrival time convenient  2999 non-null   int64 
 9   Food and drink                     2999 non-null   int64 
 10  Gate location                      2999 non-null   int64 
 11  Inflight wifi service              2999 non-null   int64 
 12  Inflig

In [4]:
train_x = train.drop(["id","target"],axis=1) 
train_y = train.target 

In [10]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Gender                             2999 non-null   object
 1   Customer Type                      2999 non-null   object
 2   Age                                2999 non-null   int64 
 3   Type of Travel                     2999 non-null   object
 4   Class                              2999 non-null   object
 5   Flight Distance                    2999 non-null   int64 
 6   Seat comfort                       2999 non-null   int64 
 7   Departure/Arrival time convenient  2999 non-null   int64 
 8   Food and drink                     2999 non-null   int64 
 9   Gate location                      2999 non-null   int64 
 10  Inflight wifi service              2999 non-null   int64 
 11  Inflight entertainment             2999 non-null   int64 
 12  Online

In [8]:
encoded_train_x = pd.get_dummies(train_x, columns=['Gender', "Customer Type", "Type of Travel", "Class"], drop_first = True)

In [9]:
encoded_train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                2999 non-null   int64
 1   Flight Distance                    2999 non-null   int64
 2   Seat comfort                       2999 non-null   int64
 3   Departure/Arrival time convenient  2999 non-null   int64
 4   Food and drink                     2999 non-null   int64
 5   Gate location                      2999 non-null   int64
 6   Inflight wifi service              2999 non-null   int64
 7   Inflight entertainment             2999 non-null   int64
 8   Online support                     2999 non-null   int64
 9   Ease of Online booking             2999 non-null   int64
 10  On-board service                   2999 non-null   int64
 11  Leg room service                   2999 non-null   int64
 12  Baggage handling    

In [ ]:
from sklearn.preprocessing import MinMaxScaler

num_features = ['Gender', 'Csutomer' 'Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes', 'Age']

scaler = MinMaxScaler()
encoded_train_x[num_features] = scaler.fit_transform(encoded_train_x[num_features]) 
encoded_train_x.info()

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [15]:
x_train, x_test, y_train, y_test = train_test_split(encoded_train_x, train_y, test_size = 0.2, random_state = 42)

xgb_model = XGBClassifier()

# StratifiedKFold를 사용한 5-Fold 교차 검증
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(xgb_model, x_train, y_train, cv=cv, scoring='accuracy')

In [16]:


# 교차 검증 정확도 출력
print(f"교차 검증 정확도: {cv_scores}")
print(f"평균 교차 검증 정확도: {cv_scores.mean()}")

# 테스트 데이터에 대한 예측
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost 정확도: {accuracy}")

accuracy1 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
print(f"XGBoost 정확도(불만족을 불만족으로 예측): {accuracy1}")

accuracy2 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])
print(f"XGBoost 정확도(만족을 비행기를 만족으로 예측): {accuracy2}")

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("XGBoost 혼동 행렬:")
print(conf_matrix)

# 분류 보고서 출력
class_report = classification_report(y_test, y_pred)
print("XGBoost 분류 보고서:")
print(class_report)


교차 검증 정확도: [0.90625    0.93125    0.92916667 0.92916667 0.93110647]
평균 교차 검증 정확도: 0.9253879610299235
XGBoost 정확도: 0.9216666666666666
XGBoost 정확도(불만족을 불만족으로 예측): 0.9219858156028369
XGBoost 정확도(만족을 비행기를 만족으로 예측): 0.9213836477987422
XGBoost 혼동 행렬:
[[260  22]
 [ 25 293]]
XGBoost 분류 보고서:
              precision    recall  f1-score   support

           0       0.91      0.92      0.92       282
           1       0.93      0.92      0.93       318

    accuracy                           0.92       600
   macro avg       0.92      0.92      0.92       600
weighted avg       0.92      0.92      0.92       600

